In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import funcs.utils as utils
import funcs.plotting as plot
import funcs.amyloid as amyloid
import scipy

In [4]:
PROCESSED_DIR = "data/processed"
RANDOM_STATE = 123

### 1. Load Data
---

In [5]:
# Raw Data
data_df = pd.read_csv(os.path.join(PROCESSED_DIR, "dataset_processed.tsv"), sep='\t', index_col=0).rename(columns=amyloid.ddict_unclean)

# Fix Dates
data_df = pd.concat([pd.to_datetime(data_df[amyloid.dates][var], format="mixed") for var in amyloid.dates], axis=1, keys=amyloid.dates).join(
    data_df.drop(amyloid.dates, axis=1)  
)

# Not imputed
X = pd.read_csv(os.path.join(PROCESSED_DIR, "AL_for_ccp_02.tsv"), sep='\t', index_col=0).rename(columns=amyloid.ddict_unclean)

# Imputed
Xi_median = pd.read_csv("data/imputed/median_qvars_01.tsv", sep="\t", index_col=0).rename(columns=amyloid.ddict_unclean)
Xi_knn = pd.read_csv("data/imputed/knn_qvars_01.tsv", sep="\t", index_col=0).rename(columns=amyloid.ddict_unclean)
Xi_mice = pd.read_csv("data/imputed/mice_qvars_05.tsv", sep="\t").rename(columns={'X24_hr_UTP':'24_hr_UTP'}).rename(columns=amyloid.ddict_unclean)

In [6]:
# Clusters from consensus cluster plus
ccp_cluster_df = pd.read_csv(os.path.join(PROCESSED_DIR,"AL_with_ccp_03.tsv"), sep="\t", index_col=0)[["cluster","itemConsensus"]].dropna()

### 2. Survival Predictons
---

In [7]:
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxPHSurvivalAnalysis

from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
)

# Initial model just with quantitative variables
random_state = 20
test_size = 0.25

In [38]:
# Drop due to high missingness
to_drop = ["Amyloid type","Secondary organ","Arrhythmia ","(Autonomic)",
           "(Peripheral)","SIFE M-component","UIFE M-component",
           "Education","Abdominal fat pad CR staining", "Bone marrow CR staining"]

Xi_mice_cat = Xi_mice.join(data_df.loc[Xi_mice.index, amyloid.catvars]).join(ccp_cluster_df['cluster']).drop(columns=to_drop).dropna()

# Collapse Race
Xi_mice_cat["Race"] = Xi_mice_cat["Race"].apply(lambda x: "Other" if x in ['American_Indian_Alaska_Native','Multiracial','Native_Hawaiian_Pacific', 'Unknown/other'] else x)

# Compute BU Staging with imputed values
Xi_mice_cat["BU Stage (Computed)"] = Xi_mice_cat.apply(lambda row: utils.assign_bu_stage(row), 1)

# Ensure Cluster is categorical
Xi_mice_cat["cluster"] = Xi_mice_cat["cluster"].astype(str)

# Split to categorical variables
n_cat = Xi_mice_cat.loc[:,Xi_mice_cat.dtypes == "object"].columns
n_numeric = Xi_mice_cat.loc[:,Xi_mice_cat.dtypes != "object"].columns

Xi_mice_cat = OneHotEncoder().fit_transform(Xi_mice_cat.loc[:,n_cat].astype("category")).join(Xi_mice_cat.loc[:,n_numeric])

print("Using {} quantitative variables.".format(n_numeric.shape[0]))
print("Using {} categorical variables.".format(n_cat.shape[0]))
print("Total samples: n={}".format(Xi_mice_cat.shape[0]))

# Join outcome variables (time, status) and cluster
Xi_mice_bu_df = Xi_mice_cat.join(data_df[["time","status"]])

Using 29 quantitative variables.
Using 35 categorical variables.
Total samples: n=1601


#### 2a. Baseline Cox Regression Models
---

In [105]:
from itertools import chain

In [106]:
test_size = 0.25
RANDOM_STATE=122

In [107]:
# Generate outcome 
y = Xi_mice_bu_df.loc[:,['status','time']]
y.loc[:,'status'] = y['status'].replace({1:True,0:False})
y = np.array(list(y.to_records(index=False)))

# Split data
X_train, X_test, y_train, y_test = train_test_split(Xi_mice_bu_df.drop(columns=['time','status']), y, test_size=test_size, random_state=RANDOM_STATE)

print("Train size (n={})".format(X_train.shape[0]))
print("Test size (n={})".format(X_test.shape[0]))

Train size (n=1200)
Test size (n=401)


In [111]:
numerical_vars = ["Age","eGFR","Troponin"]
categorical_vars = ["Sex","Race","Primary organ","BU Stage (Computed)"]
categorical_vars = list(chain(*[Xi_mice_bu_df.columns[Xi_mice_bu_df.columns.str.startswith(cat)] for cat in categorical_vars]))
vars_to_use = numerical_vars + categorical_vars

# Fit Cox Regression
cph = CoxPHSurvivalAnalysis()
cph.fit(X_train[vars_to_use], y_train)

# Predictions on Test Set
print("C-statistic (Train): ", cph.score(X_train[vars_to_use], y_train))
print("C-statistic (Test): ", cph.score(X_test[vars_to_use], y_test))

# Predictions
x_times = range(2,250,5)
cph_risk_scores = cph.predict(X_test[vars_to_use])
cph_auc, cph_mean_auc = cumulative_dynamic_auc(y_train, y_test, cph_risk_scores, x_times)

C-statistic (Train):  0.7033329440717625
C-statistic (Test):  0.6906285667699331


In [112]:
numerical_vars = ["Age","eGFR","Troponin"]
categorical_vars = ["Sex","Race","Primary organ","BU Stage (Computed)","cluster"]
categorical_vars = list(chain(*[Xi_mice_bu_df.columns[Xi_mice_bu_df.columns.str.startswith(cat)] for cat in categorical_vars]))
vars_to_use = numerical_vars + categorical_vars

# Fit Cox Regression
cph2 = CoxPHSurvivalAnalysis()
cph2.fit(X_train[vars_to_use], y_train)

# Predictions on Test Set
print("C-statistic (Train): ", cph2.score(X_train[vars_to_use], y_train))
print("C-statistic (Test): ", cph2.score(X_test[vars_to_use], y_test))

# Predictions
x_times = range(2,250,5)
cph2_risk_scores = cph2.predict(X_test[vars_to_use])
cph2_auc, cph2_mean_auc = cumulative_dynamic_auc(y_train, y_test, cph2_risk_scores, x_times)

C-statistic (Train):  0.7120264524863001
C-statistic (Test):  0.702735203000163


In [113]:
numerical_vars = []
categorical_vars = ["BU Stage (Computed)"]
categorical_vars = list(chain(*[Xi_mice_bu_df.columns[Xi_mice_bu_df.columns.str.startswith(cat)] for cat in categorical_vars]))
vars_to_use = numerical_vars + categorical_vars

# Fit Cox Regression
cph3 = CoxPHSurvivalAnalysis()
cph3.fit(X_train[vars_to_use], y_train)

# Predictions on Test Set
print("C-statistic (Train): ", cph3.score(X_train[vars_to_use], y_train))
print("C-statistic (Test): ", cph3.score(X_test[vars_to_use], y_test))

# Predictions
x_times = range(2,250,5)
cph3_risk_scores = cph3.predict(X_test[vars_to_use])
cph3_auc, cph3_mean_auc = cumulative_dynamic_auc(y_train, y_test, cph3_risk_scores, x_times)

C-statistic (Train):  0.652424018303944
C-statistic (Test):  0.6436389205935105


#### 2b. Random Survival Forest
---

In [124]:
for x in X_train.columns:
    if "BU" in x or "cluster" in x:
        print(x)

cluster=2.0
cluster=3.0
BU Stage (Computed)=stage II
BU Stage (Computed)=stage III
BU Stage (Computed)=stage IIIb


In [ ]:
# Initial RSF model, no hyperparameter search
rsf = RandomSurvivalForest(
    n_estimators=10, min_samples_split=10, min_samples_leaf=15, n_jobs=-1, random_state=RANDOM_STATE
)

# Fit Model
rsf.fit(X_train, y_train)

# Predictions on Test Set
# Compute cumulative dynamic AUC
x_times = range(2,250,5)
rsf_risk_scores = rsf.predict(X_test)
rsf_auc, rsf_mean_auc = cumulative_dynamic_auc(y_train, y_test, rsf_risk_scores, x_times)

In [ ]:
plot.plot_cumulative_dynamic_auc(x_times, rsf_auc, rsf_mean_auc)
print("C-statistic (Train): ", rsf.score(X_train, y_train))
print("C-statistic (Test): ", rsf.score(X_test, y_test))

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(rsf, X_test, y_test, n_repeats=15, random_state=random_state)

pd.DataFrame(
    {
        k: result[k]
        for k in (
            "importances_mean",
            "importances_std",
        )
    },
    index=X_test.columns,
).sort_values(by="importances_mean", ascending=False).tail()

In [19]:
###

In [20]:
# class DataObject(object):
#     """_summary_

#     Args:
#         object (_type_): _description_
#     """
#     def __init__(self, X: pd.DataFrame, time: str = "time", indicator: str = "status") -> None:
#         """_summary_

#         Args:
#             X (pd.DataFrame): _description_
#             time (str, optional): _description_. Defaults to "time".
#             indicator (str, optional): _description_. Defaults to "status".
#         """
#         from sksurv.preprocessing import OneHotEncoder

#         self.X = X

#         # Select numeric and categorical variables
#         self.X_cat = self.X.loc[:, self.X.dtypes in ["object","category"]].astype("category")
#         self.X_numeric = self.X.loc[:, self.X.dtypes not in  ["object","category"]]

#         # One hot encode categorical variables
#         self.X_cat_onehot = OneHotEncoder().fit_transform(self.X_cat)

#     def __str__(self) -> str:
#         print("Data Object \n \t * {} nategorical variables \n \t * {} numeric variables".format(
#             self.X_cat.shape[0], self.X_numeric.shape[0]))
        
# DataObject(test_df)